In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, BatchNormalization, GlobalAveragePooling2D, Input,MaxPool2D, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam, SGD,Nadam,AdamW,RMSprop


import matplotlib.pyplot as plt
import numpy as np

from sklearn.metrics import roc_curve, auc, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# Apply Cutout augmentation
def apply_cutout(image, size=8, n_holes=1):
    h, w = image.shape[0], image.shape[1]
    for n in range(n_holes):
        # Random position of cutout
        y = np.random.randint(h)
        x = np.random.randint(w)

        # Ensure cutout stays within image bounds
        y1 = np.clip(y - size // 2, 0, h)
        y2 = np.clip(y + size // 2, 0, h)
        x1 = np.clip(x - size // 2, 0, w)
        x2 = np.clip(x + size // 2, 0, w)

        # Set the cutout region to zero
        image[y1:y2, x1:x2, :] = 0
    return image

# Apply cutout to training data
x_train_ship_cutout = x_train_ship.copy()
for i in range(len(x_train_ship_cutout)):
    x_train_ship_cutout[i] = apply_cutout(x_train_ship_cutout[i])


In [2]:
# Load CIFAR-10 data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# ---- TRUCK vs NON-TRUCK ----
# Prepare balanced dataset for truck (class 9) vs non-truck
def prepare_binary_data(x, y, target_class):
    pos_idx = np.where(y == target_class)[0]
    neg_idx = np.where((y != target_class))[0]
    n = min(len(pos_idx), len(neg_idx))
    idx = np.concatenate([pos_idx[:n], np.random.choice(neg_idx, n, replace=False)])
    np.random.shuffle(idx)
    return x[idx].astype('float32') / 255.0, (y[idx] == target_class).astype(int)

x_train_truck, y_train_truck = prepare_binary_data(x_train, y_train, target_class=9)
x_test_truck, y_test_truck = prepare_binary_data(x_test, y_test, target_class=9)


170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 11s 0us/step


In [ ]:
model = tf.keras.models.load_model('ship_non_ship_cnn_model.keras')


for layer in model.layers:
    layer.trainable = False

# ---- Unfreeze only the last Conv2D and GlobalAveragePooling2D layers ----
# Find last Conv2D
last_conv2d = None
for layer in reversed(model.layers):
    if isinstance(layer, tf.keras.layers.Conv2D):
        last_conv2d = layer
        break

# Find GlobalAveragePooling2D
gap_layer = None
for layer in model.layers:
    if isinstance(layer, tf.keras.layers.GlobalAveragePooling2D):
        gap_layer = layer
        break

if last_conv2d is not None:
    last_conv2d.trainable = True
if gap_layer is not None:
    gap_layer.trainable = True

# ---- Recompile ----
model.compile(optimizer=RMSprop(learning_rate=4.38e-3, weight_decay=2e-4), loss='binary_crossentropy', metrics=['accuracy'])

# ---- Fine-tune on automobile vs non-automobile ----


def apply_cutout(image, size=8, n_holes=1):
    h, w = image.shape[0], image.shape[1]
    for n in range(n_holes):
        # Random position of cutout
        y = np.random.randint(h)
        x = np.random.randint(w)

        # Ensure cutout stays within image bounds
        y1 = np.clip(y - size // 2, 0, h)
        y2 = np.clip(y + size // 2, 0, h)
        x1 = np.clip(x - size // 2, 0, w)
        x2 = np.clip(x + size // 2, 0, w)

        # Set the cutout region to zero
        image[y1:y2, x1:x2, :] = 0
    return image
def cutout_preprocess(img):
    return apply_cutout(img)
datagen = ImageDataGenerator(

    rotation_range=12,
    width_shift_range=0.10,
    height_shift_range=0.14,
    zoom_range=0.12,
    shear_range=0.2,
    horizontal_flip=True,
    preprocessing_function=apply_cutout

)
datagen.fit(x_train_truck)

callbacks = [
    EarlyStopping(monitor='val_loss', patience=4, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.4, patience=3, verbose=1)
]

print("Fine-tuning ship model on truck vs non-truck images...")
history_truck = model.fit(
    datagen.flow(x_train_truck, y_train_truck, batch_size=8),
    epochs=20,
    validation_data=(x_test_truck, y_test_truck),
    callbacks=callbacks,
    steps_per_epoch=len(x_train_truck) // 8,
    verbose=1
)

# Save final model
model.save('truck_non_truck_cnn_model.keras')



Fine-tuning ship model on truck vs non-truck images...
Epoch 1/20


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1250/1250 ━━━━━━━━━━━━━━━━━━━━ 42s 32ms/step - accuracy: 0.5923 - loss: 0.7014 - val_accuracy: 0.6390 - val_loss: 0.6149 - learning_rate: 0.0044
Epoch 2/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 34s 27ms/step - accuracy: 0.6352 - loss: 0.6430 - val_accuracy: 0.6830 - val_loss: 0.5919 - learning_rate: 0.0044
Epoch 3/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 35s 28ms/step - accuracy: 0.6429 - loss: 0.6406 - val_accuracy: 0.6795 - val_loss: 0.5941 - learning_rate: 0.0044
Epoch 4/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 32s 25ms/step - accuracy: 0.6448 - loss: 0.6388 - val_accuracy: 0.6940 - val_loss: 0.5881 - learning_rate: 0.0044
Epoch 5/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 41s 25ms/step - accuracy: 0.6413 - loss: 0.6338 - val_accuracy: 0.6955 - val_loss: 0.5827 - learning_rate: 0.0044
Epoch 6/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 33s 27ms/step - accuracy: 0.6372 - loss: 0.6407 - val_accuracy: 0.6865 - val_loss: 0.5801 - learning_rate: 0.0044
Epoch 7/20
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 41s 26ms/step - accuracy: 0.6603 

In [ ]:
# prompt: ROC curve of truck non truck model and confusion matrix

import matplotlib.pyplot as plt
# Evaluate the fine-tuned model on the test set to get predictions for truck vs non-truck
y_pred_prob_truck = model.predict(x_test_truck).ravel()
y_pred_class_truck = (y_pred_prob_truck > 0.56).astype(int)

# ---- ROC Curve for Truck vs Non-Truck ----
fpr_truck, tpr_truck, thresholds_truck = roc_curve(y_test_truck, y_pred_prob_truck)
roc_auc_truck = auc(fpr_truck, tpr_truck)

plt.figure()
plt.plot(fpr_truck, tpr_truck, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc_truck)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic for Truck/Non-Truck Classification')
plt.legend(loc="lower right")
plt.show()

# ---- Confusion Matrix for Truck vs Non-Truck ----
cm_truck = confusion_matrix(y_test_truck, y_pred_class_truck)
disp_truck = ConfusionMatrixDisplay(confusion_matrix=cm_truck, display_labels=['Non-Truck', 'Truck'])
disp_truck.plot(cmap=plt.cm.Blues)
plt.title('Confusion Matrix for Truck/Non-Truck Classification')
plt.show()